# NRC-30 NIRCam Coronagraphic Target Acquisition

## PID 775 - Partial Rehearsal Version

This activity is a two-part test of NIRCam coronagraphic target acquisition (TA).  Part 1 determines the precise offsets required to move a star from its measured position in the target-acquisition image to the ideal location behind each occulter.   Part 2 follows the analysis period for Part 1 and tests the implementation of the derived offsets. Both parts follow the successful completion of CAR-NIRCAM-021b (Coronagraphic Astrometric Calibration), the goal of which is a geometric distortion map of the coronagraphic FOV with RMS errors less than 3 mas in each axis.

In [1]:
# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# Enable inline plotting at lower left
%matplotlib inline

# Load progress bars
from tqdm.auto import trange, tqdm

In [2]:
import pynrc
from pynrc import nrc_utils, obs_hci, nrc_hci

# Disable informational messages and only include warnings and higher
pynrc.setup_logging(level='WARN')

pyNRC log messages of level WARN and above will be shown.
pyNRC log outputs will be directed to the screen.


In [3]:
# Observed spectrum
bp_k = pynrc.bp_2mass('k')
sp = pynrc.stellar_spectrum('A0V', 5.75, 'vegamag', bp_k)

# Science and reference information
params_hci_src = {'sp_sci': sp, 'sp_ref': sp, 'dist_sci': 10}

# Companions
params_hci_companions = None
# Disk model
params_disk_model = None
# Sources
src_tbl = None

In [4]:
pid_str = 'pid1075'
nrc_str = 'NRC30'

sim_config = {
    # APT input files
    'json_file'     : f'{pid_str}.timing.json',
    'sm_acct_file'  : f'{pid_str}.smart_accounting',
    'pointing_file' : f'{pid_str}.pointing',
    'xml_file'      : f'{pid_str}.xml',
    # Output directory
    'save_dir'      : f'/Users/jarron/NIRCam/Data/Sim_{nrc_str}/',
    
    # Date and time of observations
    'obs_date' : '2022-03-01', 
    'obs_time' : '12:00:00',
    # Position angle of observatory
    # User should check acceptable range in APT's Roll Analysis
    'pa_v3'    : 0,
    
    # Source information
    'src_tbl'               : src_tbl,  # astropy table with RA, Dec, and filter mags
    'params_hci_src'        : params_hci_src,
    'params_disk_model'     : params_disk_model,
    'params_hci_companions' : params_hci_companions,
    
    # PSF size information for WebbPSF_ext
    'params_webbpsf' : {'fov_pix': 257, 'oversample': 2},
    # Position-dependent PSFs for convolution
    'params_psfconv' : {'npsf_per_full_fov': 5, 'osamp': 1, 'sptype': 'G0V'},
    # Wavefront error drift settings
    'enable_wfedrift' : False,
    # For coronagraphic masks, sample large grid of points?
    'large_grid' : True,
    
    # Slew and dither pointing uncertainties
    'large_slew' : 100.0,   # Slew to target (mas)
    'ta_sam'     : 5.0,     # SAM movements from TA position (mas)
    'std_sam'    : 5.0,     # Standard dither values (mas)
    'sgd_sam'    : 2.5,     # Small grid dithers (mas)
    
    # Type of image files to save
    'save_slope' : False,  # Save ideal noiseless slope images to FITS
    'save_dms'   : False,   # Save DMS-like ramps to FITS
    'dry_run'    : False,  # Perform a dry-run, not generating any data, just printing visit info
    
    # Initialize random seeds if repeatability is required
    # Create separate random number generators for dithers and noise
    'rand_seed_init'  : 1234,

    # Ramp parameters
    'params_noise' : {
        'include_poisson'   : True,     # Photon Noise
        'include_dark'      : True,     # Dark current
        'include_bias'      : True,     # Bias image offset
        'include_ktc'       : True,     # kTC Noise
        'include_rn'        : True,     # Read Noise
        'include_cpink'     : True,     # Correlated 1/f noise between channel
        'include_upink'     : True,     # Channel-dependent 1/f noise
        'include_acn'       : True,     # Alternating column noise
        'apply_ipc'         : True,     # Interpixel capacitance
        'apply_ppc'         : True,     # Post-pixel coupling
        'amp_crosstalk'     : True,     # Amplifier crosstalk
        'include_refoffsets': True,     # Reference offsets
        'include_refinst'   : True,     # Reference pixel instabilities
        'include_colnoise'  : True,    # Transient detector column noise
        'add_crs'           : True,    # Include cosmic ray
        'cr_model'          : 'SUNMAX', # Cosmic ray model ('SUNMAX', 'SUNMIN', or 'FLARES')
        'cr_scale'          : 1,        # Cosmic ray probabilities scaling
        'apply_nonlinearity': True,     # Apply non-linearity
        'random_nonlin'     : True,     # Add randomness to non-linearity
        'apply_flats'       : True,     # pixel-to-pixel QE variations and field-dep illum
    },
}

In [5]:
from pynrc.simul.ngNRC import create_level1b_FITS

In [7]:
create_level1b_FITS(sim_config, visit_id='001:001', dry_run=True, save_dms=True)

Obs Params:   0%|          | 0/170 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

NRCA2 NRCA2_FULL_MASK210R F210M 001:001 00003 [-0.00483371  0.00591349]
Saving: /Users/jarron/NIRCam/Data/Sim_NRC30/pynrc_jw01075001001_01101_00003_nrca2_uncal.fits
NRCA2 NRCA2_FULL_MASK210R F210M 001:001 00004 [-0.00483371  0.00591349]
Saving: /Users/jarron/NIRCam/Data/Sim_NRC30/pynrc_jw01075001001_01101_00004_nrca2_uncal.fits
NRCA2 NRCA2_FULL_MASK210R F210M 001:001 00005 [-0.00669587  0.0285696 ]
Saving: /Users/jarron/NIRCam/Data/Sim_NRC30/pynrc_jw01075001001_01101_00005_nrca2_uncal.fits
NRCA2 NRCA2_FULL_MASK210R F210M 001:001 00006 [-0.02269406  0.02042168]
Saving: /Users/jarron/NIRCam/Data/Sim_NRC30/pynrc_jw01075001001_01101_00006_nrca2_uncal.fits
NRCA2 NRCA2_FULL_MASK210R F210M 001:001 00007 [-0.02250992  0.00440352]
Saving: /Users/jarron/NIRCam/Data/Sim_NRC30/pynrc_jw01075001001_01101_00007_nrca2_uncal.fits
NRCA2 NRCA2_FULL_MASK210R F210M 001:001 00008 [-0.02090377 -0.00612831]
Saving: /Users/jarron/NIRCam/Data/Sim_NRC30/pynrc_jw01075001001_01101_00008_nrca2_uncal.fits
NRCA2 NRCA

  0%|          | 0/12 [00:00<?, ?it/s]

NRCA2 NRCA2_FULL_TAMASK210R F210M 001:001 00002 [ 0.00101514 -0.00086851]
Saving: /Users/jarron/NIRCam/Data/Sim_NRC30/pynrc_jw01075001001_01101_00002_nrca2_uncal.fits


  0%|          | 0/12 [00:00<?, ?it/s]

NRCA2 NRCA2_TAMASK210R F210M 001:001 00001 [ 0.0203027  -0.01737015]
Saving: /Users/jarron/NIRCam/Data/Sim_NRC30/pynrc_jw01075001001_01101_00001_nrca2_uncal.fits


In [7]:
from pynrc import nrc_utils, nircam_cal, DMS_input, DetectorOps
from pynrc.simul.apt import get_detname

In [13]:
pid_str = 'pid1075'

save_dir = f'/Users/jarron/NIRCam/Data/Sim_{pid_str}/'

# APT input files
json_file     = f'{pid_str}.timing.json'
sm_acct_file  = f'{pid_str}.smart_accounting'
pointing_file = f'{pid_str}.pointing'
xml_file      = f'{pid_str}.xml'

In [14]:
# Create DMS Input class
obs_input = DMS_input(xml_file, pointing_file, json_file, sm_acct_file, save_dir=save_dir)

# # Update observing start date/time and V3 PA
obs_input.obs_date = '2022-11-03'
obs_input.obs_time = '12:00:00'

# Generate all observation parameters for every visit, exposure, detector, etc
obs_params_all = obs_input.gen_all_obs_params()
obs_params_all = np.asarray(obs_params_all)

Obs Params:   0%|          | 0/170 [00:00<?, ?it/s]

In [15]:
obs_detnames = np.array([get_detname(par['detector']) for par in obs_params_all])
obs_filters  = np.array([par['filter'] for par in obs_params_all])
obs_pupils   = np.array([par['pupil'] for par in obs_params_all])
obs_apnames  = np.array([par['siaf_ap'].AperName for par in obs_params_all])
obs_visitids = np.array([par['visit_key'] for par in obs_params_all])

obs_labels  = np.array([f'{a}_{f}' for a, f in zip(obs_apnames, obs_filters)])

In [17]:
obs_params_all[0]

{'pi_name': 'Julien Girard',
 'title': 'NIRCam Target Acquisition for Coronagraphy',
 'category': 'GO',
 'sub_category': 'UNKNOWN',
 'science_category': 'Planets and Planet Formation',
 'target_name': 'HD-42525',
 'catalog_name': 'UNKNOWN',
 'ra': 91.53909,
 'dec': -66.03962,
 'pa_v3': 308.72500767471627,
 'roll_offset': 0,
 'solar_elong': 89.8530992132778,
 'pitch_ang': -0.14690078672219897,
 'siaf_ap': <pysiaf.Aperture object AperName=NRCA2_TAMASK210R >,
 'ra_obs': 91.53909,
 'dec_obs': -66.03962000000001,
 'siaf_ap_ref': <pysiaf.Aperture object AperName=NRCA2_TAMASK210R >,
 'ra_ref': 91.53909,
 'dec_ref': -66.03962000000001,
 'date-obs': '2022-11-03',
 'time-obs': '12:00:00',
 'obs_id_info': {'program_number': '01075',
  'observation_number': '001',
  'visit_number': '001',
  'visit_group': '01',
  'sequence_id': '1',
  'activity_id': '01',
  'exposure_number': '00001',
  'visit_id': '01075001001',
  'obs_id': 'V01075001001P0000000001101'},
 'obs_label': 'MASK210R, F210M, 9-PT CIRCL